<a href="https://colab.research.google.com/github/amosqueraf/ML/blob/master/kernel_Keras_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!echo '{"username":"amosqueraf","key":"5c1cd3a8df48572877c8df1fcc57658f"}' > ~/.kaggle/kaggle.json
!kaggle competitions download -c jigsaw-unintended-bias-in-toxicity-classification -f test.csv -p ../input
!kaggle competitions download -c jigsaw-unintended-bias-in-toxicity-classification -f train.csv -p ../input
!kaggle competitions download -c jigsaw-unintended-bias-in-toxicity-classification -f sample_submission.csv -p ../input

 74% 9.00M/12.1M [00:00<00:00, 19.4MB/s]
100% 12.1M/12.1M [00:00<00:00, 21.9MB/s]
 98% 268M/273M [00:01<00:00, 168MB/s]
100% 273M/273M [00:01<00:00, 146MB/s]
  0% 0.00/221k [00:00<?, ?B/s]
100% 221k/221k [00:00<00:00, 61.9MB/s]


In [5]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output

['sample_submission.csv.zip', 'train.csv.zip', 'test.csv.zip']


In [6]:
from keras.preprocessing import text, sequence
from keras import backend as K
from keras.models import load_model, Sequential
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM
from keras.layers.core import Dense, Dropout
import keras
import pickle
from joblib import Parallel, delayed
import string
from sklearn.feature_extraction.text import TfidfVectorizer

print(K.tensorflow_backend._get_available_gpus())

Using TensorFlow backend.


[]


In [7]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
stop_words = set(stopwords.words('english'))
stem = SnowballStemmer('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [0]:
train_df = pd.read_csv("../input/train.csv.zip")
test_df = pd.read_csv("../input/test.csv.zip")

train_df = train_df[['id','comment_text', 'target']]

In [9]:
train_df.head()

,id,comment_text,target
0,59848,"This is so cool. It's like, 'would you want yo...",0.000000
1,59849,Thank you!! This would make my life a lot less...,0.000000
2,59852,This is such an urgent design problem; kudos t...,0.000000
3,59855,Is this something I'll be able to install on m...,0.000000
4,59856,haha you guys are a bunch of losers.,0.893617


In [10]:
train_df.shape, test_df.shape

((1804874, 3), (97320, 2))

In [0]:
def tokenize(text):
    
    tokens = []
    for token in word_tokenize(text):
        if token in string.punctuation: continue
        if token in stop_words: continue
        tokens.append(stem.stem(token))
    
    return " ".join(tokens)

In [0]:
train_tokens = Parallel(n_jobs=-1, verbose=1)(delayed(tokenize)(text) for text in train_df['comment_text'].tolist())

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done 5743 tasks      | elapsed:    7.8s
[Parallel(n_jobs=-1)]: Done 42343 tasks      | elapsed:   38.4s
[Parallel(n_jobs=-1)]: Done 103343 tasks      | elapsed:  1.5min


In [0]:
test_tokens = Parallel(n_jobs=-1, verbose=1)(delayed(tokenize)(text) for text in test_df['comment_text'].tolist())

In [0]:
tokenizer = TfidfVectorizer(stop_words='english')
tokenizer.fit(train_tokens + test_tokens)

In [0]:
train_data = train_df["comment_text"]
label_data = train_df["target"]
test_data = test_df["comment_text"]

In [0]:
train_data = tokenizer.transform(train_tokens)
label_data = train_df["target"]
test_data =  tokenizer.transform(test_tokens)

In [0]:
train_data.shape, label_data.shape

In [0]:
embed_dim = 256
lstm_out = 200
batch_size = 1000

model = Sequential()
model.add(Embedding(2500, embed_dim,input_length = train_data.shape[1]))
model.add(keras.layers.SpatialDropout1D(0.2))
model.add(LSTM(lstm_out, dropout=0.25, recurrent_dropout=0.28))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss = 'binary_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

In [0]:
model.fit(train_data, label_data, batch_size = batch_size, epochs = 1,  verbose = 5)

In [0]:
model = LSTMModel().model
model.fit(
    train_data, 
    label_data, 
    batch_size = BATCH_SIZE, 
    epochs = NUM_EPOCH)

In [0]:
result = model.predict(test_data)

In [0]:
submisson_df = pd.read_csv("..../input/sample_submission.csv")
submisson_df['prediction'] = result
submisson_df['prediction'] = submisson_df['prediction'].apply(lambda x: "%.5f" % x if x > 0 else 0.0)
submisson_df.to_csv("submission.csv", index=False)

In [0]:
submission.head()